In [ ]:
import pandas as pd
import numpy as np

df=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",header=None)

names=['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','salary']

df.columns=names

Missing data is replaced by mode of the column

In [ ]:
list0=[]
for column in df.columns:
    try:
        data= df[df[column]==' ?']
        if data.shape[0]>0:
            list0.append(column)
            
    except:
        continue;

for i in list0:
    s=pd.Series(df[i])
    s.replace(' ?',s.mode()[0], inplace=True)

Categorical and Numerical data separated

In [ ]:
dfNum=df[df._get_numeric_data().columns]
dfCat=df[list(set(names)-set(df._get_numeric_data().columns))]

One Hot Encoding used for converting categorical data to numerical 
Each distinct value of an attribute is a new atrribute, for each row if old attribute value equal to new distinct arrtibute, that new distinct attribute's value is 1 else 0 for that row 

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohc=OneHotEncoder()
ohc.fit(dfCat.drop(columns='salary'))
data=ohc.transform(dfCat.drop(columns='salary')).toarray()
dfEncoded=pd.DataFrame(data, index = dfCat.index.values, columns=ohc.get_feature_names())
dfEncoded['salary']=df['salary']

Numerical data and One Hot Encoded data are combined according to index

In [ ]:
df1=pd.concat([dfNum,dfEncoded],axis=1)

Training and testing data split 5 times, so each row of original data is once in test data prediction of each test data ouput combined according to test_index and compared with output of original data output values for accuracy

Data is scaled so each attribute has significant contribution in finding euclidean distances between single test instance and every training instance

Pairwise Euclidean distances are calculated between single test instance/row and every training instance/row
Outputs of the k least distant training rows according to Euclidean distance from a test instance is considered and the majority output is the predicted output of the test instance

Algorithm is run for 4 of the k values

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import pairwise_distances

from sklearn.preprocessing import StandardScaler

acc=[]
for k in range(1,4):    
    results=pd.Series()
    for train_index, test_index in kf.split(df1):
        Xtrain, Xtest, Ytrain, Ytest = df1.drop(columns='salary').iloc[train_index], df1.drop(columns='salary').iloc[test_index], df1['salary'].iloc[train_index], df1['salary'].iloc[test_index]
        s = StandardScaler()
        
        Xtrain=Xtrain.astype(float)
        Xtest=Xtest.astype(float)

        Xtrain=s.fit_transform(Xtrain)
        Xtest=s.transform(Xtest)
        
        Dist=pairwise_distances(Xtest, Xtrain, metric='euclidean')
        
        predict=[]
        for i in range(0,Xtest.shape[0]):
            sorting=pd.Series(Dist[i],index=train_index)
            sorting=sorting.sort_values()
            Output=sorting[0:k]
    
            O=Ytrain[Output.index[0:k]]
        
            predict.append(max(O.value_counts().index))
            
        res=pd.Series(predict,index=test_index)
        results=results.append(res)
        
    results=results.sort_index()
    
    from sklearn import metrics
    print("Accuracy:",(metrics.accuracy_score(df1['salary'], results)*100),"%",k)
    acc.append(metrics.accuracy_score(df1['salary'], results))
    
print('Accuracy:',(max(acc))*100,'%','k=',acc.index(max(acc)))        